# <center>Critical AI</center>
<center>ENGL 54.41</center>
<center>Dartmouth College</center>
<center>Fall 2024</center>

<center><h2>Midterm Exam</h2></center>
<hr>
<pre>Created: 10/11/2024</pre>


Due: 11:59pm Wednesday, October 23rd.

In [ ]:
import numpy as np
import random
from glob import glob
from matplotlib import pyplot as plt
from PIL import Image
import numpy, os, re
import torch, torchvision
from torchvision.models import vgg16
from torchvision.io import read_image, ImageReadMode
import sklearn

from sklearn.decomposition import PCA

**Question #1**: Critical AI

What are the meaningful differences and similarities between two different (select accounts from different essays/chapters) definitions of "critical AI" found in our readings thus far? (Approx. 500 words)

**Question #2**: On Neural networks and learning. 

Using Frank Rosenblatt's own account of the Perceptron in “Design of an Intelligent Automaton,” explain what he means by learning. Make sure to quote and cite his article. How does this approach compare with the everyday understanding of learning? (Approx. 250 words)

**Question #3**: Generalization in Convolutional Neural Networks. Explain how you understand generalization to work in convolutional neural networks and the differences and similarities of generalization to instrumental realism, as presented by Malevé and Sluis. (Approx 250 words)


##### **Question #4**: Critique of Image Pipeline

In Emily Denton, et al., “On the Genealogy of Machine Learning Datasets: A Critical History of ImageNet,” *Big Data & Society* 8, no. 2 (2021), the authors write: “We analyze discourses which shaped ImageNet, focusing on three problems: the importance of data; meaning and the computational construction of understanding; and the strategic choices regarding the visibility (and invisibility) of labor." Please first explain the significance and relation of these three elements and then apply this critical lens to your reading of the ["Deep Learning Face Attributes in the Wild"](https://liuziwei7.github.io/projects/FaceAttributes.html) paper and to the [CelebA dataset](https://mmlab.ie.cuhk.edu.hk/projects/CelebA.html). (Approx 500 words)

**Question #5**: Applying Critique to VGG16.

Upload an image, run the cells below, and explain in the markdown cell how and where all three problems mentioned above are found in the output of this pre-trained VGG16 model. (Approx. 250 words)

In [ ]:
model = vgg16(weights='DEFAULT')
weights = torchvision.models.VGG16_Weights.DEFAULT
preprocess = weights.transforms()

def display_image(image_file):
    img = np.asarray(Image.open(image_file))
    plt.imshow(img)
    
def decode_predictions(prediction,topk):
    # can't figure out how to make tensor's argsort reverse like this
    # we should already be on the CPU but leaving as such for future use
    class_ids = np.argsort(prediction.to('cpu').detach().numpy())[::-1][:topk]
    for i in class_ids:
        score = prediction[i].item()
        category_name = weights.meta["categories"][i]
        print(f"{100 * score:5.2f}% {category_name}")

def get_prediction(image_file,topk=5,display_flag=False):
    img = read_image(image_file,mode=ImageReadMode.RGB)
    batch = preprocess(img).unsqueeze(0)
    # we should already be on the CPU but leaving as such for future use
    prediction = model(batch.to('cpu')).squeeze(0).softmax(0)
    if display_flag:
        display_image(image_file)
    decode_predictions(prediction,topk=topk)

In [ ]:
get_prediction("image.jpg", display_flag=True)

**Question #6**: Vector Space Similarities in Text Models.

First modify the values of the list "concept_a" and "concept_b" to contain words that you believe to be associate with two different concepts. These should be single words and lowercase. Use fairly basic terms. You should have a minimum of five words in each list. What happens if you add words that seem like they could belong to either concept? Then, referencing Mitchell and Gavin, explain the meaning of the resulting scatterplot (Approx. 250 words).

In [ ]:
# download model
import gensim.downloader as api
google_model = api.load("word2vec-google-news-300")
vocab_size, dim = google_model.vectors.shape

print(f"vocab size: {vocab_size}, vector width: {dim}")

concept_a = ["word1","word2","word3","word4","word5"]
concept_b = ["word1","word2","word3","word4","word5"]

vecs, terms = list(), list()
for w in concept_a + concept_b:
    if w in google_model:
        vecs.append(google_model[w])
        terms.append(w)

pca = PCA(n_components=2)
plot_data = pca.fit_transform(vecs)
xs, ys = plot_data[:, 0], plot_data[:, 1]

fig = plt.figure(figsize=(20, 15))
plt.title("Embedding Space: Concept A + B")
plt.scatter(xs, ys, marker = '^')
for i, w in enumerate(terms):
    plt.annotate(w, xy = (xs[i], ys[i]), xytext = (3, 3),
                 textcoords = 'offset points', ha = 'left', va = 'top')
plt.show()  